In [1]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

/tmp/ipykernel_662113/1628820390.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv('./in/copper_returns_5d_final.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

In [3]:
def fit_evaluate_sarima(series, order, seasonal_order):
    # Dividir en conjuntos de entrenamiento y prueba
    train, test = train_test_split(series, test_size=0.2, shuffle=False)

    # Ajustar el modelo SARIMA
    model = SARIMAX(train, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
    fitted_model = model.fit(disp=False)

    # Predicciones
    predictions = fitted_model.get_forecast(steps=len(test))
    predicted_means = predictions.predicted_mean

    # Calcular el RMSE
    rmse = np.sqrt(mean_squared_error(test, predicted_means))

    return fitted_model.summary(), rmse

In [4]:
summary, rmse = fit_evaluate_sarima(data['LMCADY_acu_5d'], (1, 0, 1), (0, 1, 1, 12))

/home/fochoa/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/fochoa/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/fochoa/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/fochoa/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/foch

In [5]:
print(summary)
print("RMSE:", rmse)

                                     SARIMAX Results                                      
Dep. Variable:                      LMCADY_acu_5d   No. Observations:                 4431
Model:             SARIMAX(1, 0, 1)x(0, 1, 1, 12)   Log Likelihood               10661.369
Date:                            Tue, 16 Apr 2024   AIC                         -21314.738
Time:                                    17:59:55   BIC                         -21289.176
Sample:                                         0   HQIC                        -21305.722
                                           - 4431                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.7670      0.008     97.047      0.000       0.751       0.782
ma.L1          0.0299      0.012   